In [1]:
import tensorflow as tf
import keras
from keras.layers import Input ,Dense, Dropout, Activation, LSTM
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Reshape, BatchNormalization, Add
from keras.models import Sequential
from keras.layers.wrappers import TimeDistributed
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.models import Model
from keras.optimizers import Adam
from keras.engine.network import Network
from keras import backend as K
from keras.models import model_from_json

import keras.backend.tensorflow_backend as KTF

from keras.initializers import glorot_normal,orthogonal

from keras.callbacks import EarlyStopping,TensorBoard
import csv

from PIL import Image

%matplotlib inline

import matplotlib.pyplot as plt

import numpy as np
import cv2
import os
import random

from tqdm import tqdm

# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.4
# sess = tf.Session(config=config)
# K.set_session(sess)


config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.visible_device_list = "0"
K.set_session(tf.Session(config=config))

Using TensorFlow backend.


In [2]:
#どんくらいで学習させるか関連
timesteps=10 #一回のLSTMに入れる値の数
camera=5 #カメラの数

rate = 10 #飛ばすフレーム数（30FPSを30/rateのFPSの動画に疑似変換する．）

#画像関連 
channels=3
img_width=50
img_height=50

#LSTMなどで用いる値
epochs = 100 # エポック数 
batch_size = 5 # バッチサイズ

In [3]:
from keras.models import load_model
model = load_model('model/file0206.tf', compile=False)
print(model.summary())












Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 10, 50, 50, 3 0                                            
__________________________________________________________________________________________________
input_10 (InputLayer)           (None, 10, 50, 50, 3 0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           (None, 10, 50, 50, 3 0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           (None, 10, 50, 50, 3 0                            

In [4]:
class DataLoader2:
    def __init__(self, camera_num, file_name, timesteps, width, height):
        self.camera_num = camera_num
        self.file_name =file_name
        self.samples=0
        self.timesteps=timesteps
        self.width=width
        self.height=height
        self.caps=[]
        self.save_numpy()
        
    def save_numpy(self):
        for i in range(0,self.camera_num):
            video_path = self.file_name+"video/"+str(i)+".mp4"
            self.caps.append(cv2.VideoCapture(video_path))
            
        self.video_len = int(self.caps[0].get(cv2.CAP_PROP_FRAME_COUNT))
            
        if not (os.path.isfile(self.file_name+"test/test_"+str(rate)+".npy")):
            self.make_numpy_testData()
        self.learnData=np.load(self.file_name+"test/test_"+str(rate)+".npy")
        print(self.learnData.shape[1])
        
        
    def make_numpy_testData(self):
        learnData=[]
        for i in range(0, self.camera_num):
            print()
            img_list=[]
            for j in range(0,self.video_len):
                
                pro_size=20
                bar = int(j*pro_size/self.video_len)
                pro_bar = ('=' * bar) + (' ' * (pro_size - bar))
                percent ='{:03f}'.format(j / self.video_len * 100.)
                print('\r{0}/{1} [{2}] {3}%'.format((i+1), self.camera_num, pro_bar, percent), end='')
                
                ret, frame = self.caps[i].read()
                if(j%rate==0):
                    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    img=Image.fromarray(frame)
                    img = img.resize((self.width, self.height))
                    x = np.array(img, dtype=np.float32)
                    x = x / 255.
                    img_list.append(x)
            learnData.append(img_list)
            
        tmp_list=np.array(learnData)
        tmp_list=tmp_list.astype(np.float)
        np.save(self.file_name+"test/test_"+str(rate)+".npy", tmp_list)
        print(tmp_list.shape)
    
        
    def make_testdata(self):
        data_name=self.file_name+"test/test_0_"+str(rate)+".npy"
        if (os.path.isfile(data_name)):
            test=np.load(data_name)
        else:
            all_list = [np.empty((0,self.timesteps, self.width, self.height, 3), np.float)]*self.camera_num
            tmp_list = [np.empty((0,self.timesteps, self.width, self.height, 3), np.float)]*self.camera_num
            for i in range(0, self.learnData.shape[1]-self.timesteps +1):
                
                #まずはすべての分だけ回す
                pro_size=20
                bar = int(i*pro_size/self.learnData.shape[1])
                pro_bar = ('=' * bar) + (' ' * (pro_size - bar))
                percent ='{:03f}'.format(i / self.learnData.shape[1] * 100.)
                print('\r [{0}] {1}%'.format(pro_bar, percent), end='')
                
                for camera in range(0, self.camera_num):
                    #カメラの台数だけ回す
                    #tmp_list[camera]の同じ引数（番号）には角度を同じ分だけずらした同じ時間軸のsequenceが入っている．
                    first_list =np.empty((0, self.width, self.height, 3), np.float)
                    for l in range(0, self.timesteps):
                        #timesteps分のndarrayを作る．
                        img=self.learnData[camera,i+l]
                        first_list =np.append(first_list, [img], axis=0)
                    #あるカメラについて，１つのsequenceがfirst_listに出来上がっている状態．
                    tmp_list[camera]=np.append(tmp_list[camera], [first_list], axis=0)
                    
                if(tmp_list[0].shape[0]>100 or i == self.learnData.shape[1]-self.timesteps):
                    #処理時間短縮のため，sequenceがtmp_listに溜まってきたら，全体に統合してまたtmp_listを初期化する．
                    for camera in range(0, self.camera_num):
                        all_list[camera]=np.append(all_list[camera], tmp_list[camera], axis=0)
                        tmp_list[camera] = np.empty((0,self.timesteps, self.width, self.height, 3), np.float)
            
            for camera in range(0, self.camera_num):
                print(all_list[camera].shape)
                data_name=self.file_name+"test/test_"+str(camera)+"_"+str(rate)
                np.save(data_name, all_list[camera])
    
    
    def get_testdata(self, num):
        data_name=self.file_name+"test/test_"+str(num)+"_"+str(rate)+".npy"
        test=np.load(data_name)
        return test
    
    def get_image(self, camera_num, frame_num):    
        self.caps[camera_num].set(cv2.CAP_PROP_POS_FRAMES, frame_num)
        ret, frame = self.caps[camera_num].read()
        return frame
    
    
    def make_labelData(self):
        labelData=[]
        test=[0,0,0,0,0]
        csv_selection=csv.reader(open(self.file_name+"video/mintime_optimize.csv", 'r'))
        one_hot = np.eye(self.camera_num)
        for i,row2 in enumerate(csv_selection):
            if(i%rate==0 and i > (self.timesteps-1)*rate-1):
                labelData.append(one_hot[int(row2[1])])
                test[int(row2[1])]+=1
        tmp_list = np.array(labelData)
        tmp_list = tmp_list.astype(np.float)
        np.save(self.file_name+"test/label_"+str(rate)+".npy", tmp_list)
        print(tmp_list.shape)
    
    def get_labeldata(self):
        data_name=self.file_name+"test/label_"+str(rate)+".npy"
        test=np.load(data_name)
        return test
    
    def show(self):
        print(0,self.get_testdata(0).shape)
        print(1,self.get_testdata(1).shape)
        print(2,self.get_testdata(2).shape)
        print(3,self.get_testdata(3).shape)
        print(4,self.get_testdata(4).shape)
        print("label", self.get_labeldata().shape)

In [8]:
data = DataLoader2(camera, "./data/20190426/", timesteps, img_width, img_height)
data.make_testdata()
data.make_labelData()
data.show()

test_learn=[np.empty((data.get_testdata(0).shape[0],10, 50, 50, 3), np.float)]*camera
for j in range(0,camera):
    test_learn[j]=data.get_testdata(j)

print(test_learn[0].shape)

376
(367, 5)
0 (367, 10, 50, 50, 3)
1 (367, 10, 50, 50, 3)
2 (367, 10, 50, 50, 3)
3 (367, 10, 50, 50, 3)
4 (367, 10, 50, 50, 3)
label (367, 5)
(367, 10, 50, 50, 3)


In [9]:
# 正答率集計
preds = model.predict([test_learn[0], test_learn[1], test_learn[2], test_learn[3], test_learn[4]])
correct = 0

tmp_list=[]

for i in range(0, len(preds[1])):
    a = [preds[0][i][0], preds[1][i][0], preds[2][i][0], preds[3][i][0], preds[4][i][0]]
    tmp_list.append(a)
    
result_array=np.array(tmp_list)
result_array=result_array.astype(np.float)
print(result_array.shape)

f = open("./data/20190427/test/result.csv", 'w')
f1 = open("./data/20190427/test/result1.csv", 'w')
writer = csv.writer(f)
writer1 = csv.writer(f1)

for i in range(0,timesteps*rate):
    writer.writerow([0])
    writer1.writerow([0,0,0,0,0])
count=0
correct=0
for i in range(len(result_array)):
    pred_result = np.argmax(result_array[i,:])
    tar = np.argmax(data.get_labeldata()[i,:])
    if(pred_result==tar):
        correct=correct+1
    count=count+1
    for j in range(0,rate):
        writer.writerow([pred_result])
        writer1.writerow(result_array[i,:])
result=correct/count
print("Accuracy: ", result)

(367, 5)
Accuracy:  0.5258855585831063


In [7]:
fourcc = cv2.VideoWriter_fourcc('m','p','4','v')
video = cv2.VideoWriter('./data/20190427/test/video.mp4', fourcc, 30.0, (640, 540))

with open('./data/20190427/test/result.csv') as f:
    reader = csv.reader(f)
    num=len(result_array)*rate
    i=0
    for row in reader:
        
        pro_size=20
        bar = int(i*pro_size/num)
        pro_bar = ('=' * bar) + (' ' * (pro_size - bar))
        percent ='{:03f}'.format(i / num * 100.)
        print('\r [{0}] {1}%'.format(pro_bar, percent), end='')
        
        img=data.get_image(int(row[0]), i)
        if not img is None:
            video.write(img)
            i+=1

video.release()

 [================    ] 84.754811%